# 에이전트 중심 워크플로우 테스트

이 노트북은 Text-to-SQL 시스템의 에이전트 중심 워크플로우를 테스트하기 위한 것입니다.
기존의 2단계 호출 방식(스키마 조회 -> 쿼리 실행)을 단일 에이전트 호출로 개선하는 것을 검증합니다.

In [2]:
import sys
import os
from pathlib import Path

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = Path.cwd().parent
print(project_root)
sys.path.append(str(project_root))

from backend.core.agent import TextToSQLAgent
from backend.core.tools.schema_analyzer import SchemaAnalyzerTool
from backend.core.tools.sql_executor import SQLExecutionTool
from backend.core.tools.sql_generator import SQLGeneratorTool
from backend.core.tools.query_analyzer import QueryAnalyzerTool
from backend.core.tools.result_formatter import ResultFormatterTool

/home/arkwith/Dev/LLM/text-to-sql/backend


ModuleNotFoundError: No module named 'backend'

## 1. 에이전트 초기화

필요한 모든 도구들을 포함한 에이전트를 초기화합니다.

In [ ]:
# 도구들 초기화
schema_analyzer = SchemaAnalyzerTool()
sql_executor = SQLExecutionTool()
sql_generator = SQLGeneratorTool()
query_analyzer = QueryAnalyzerTool()
result_formatter = ResultFormatterTool()

# 에이전트 초기화
agent = TextToSQLAgent(
    tools=[
        schema_analyzer,
        sql_executor,
        sql_generator,
        query_analyzer,
        result_formatter
    ]
)

## 2. 테스트 쿼리 실행

에이전트가 자동으로 스키마를 분석하고 SQL을 생성하여 실행하는 과정을 테스트합니다.

In [ ]:
# 테스트할 자연어 질문들
test_queries = [
    "2023년에 가장 많은 매출을 올린 상위 5개 제품은 무엇인가요?",
    "서울 지역의 고객 중 VIP 등급인 고객의 평균 구매 금액은 얼마인가요?",
    "지난 달에 가장 많이 팔린 카테고리별 제품 수를 보여주세요."
]

# 각 질문에 대해 에이전트 실행
for query in test_queries:
    print(f"\n질문: {query}")
    print("-" * 50)
    
    # 에이전트 실행
    result = agent.run(query)
    
    # 결과 출력
    print(f"생성된 SQL:\n{result['sql']}\n")
    print(f"실행 결과:\n{result['result']}\n")
    print(f"설명:\n{result['explanation']}\n")

## 3. 에이전트의 내부 동작 분석

에이전트가 각 단계에서 어떻게 동작하는지 자세히 살펴봅니다.

In [ ]:
def analyze_agent_workflow(query):
    print(f"\n질문: {query}")
    print("-" * 50)
    
    # 1. 스키마 분석 단계
    print("1. 스키마 분석 단계")
    schema_info = schema_analyzer.analyze()
    print(f"분석된 테이블 수: {len(schema_info['tables'])}")
    
    # 2. 쿼리 분석 단계
    print("\n2. 쿼리 분석 단계")
    query_analysis = query_analyzer.analyze(query, schema_info)
    print(f"필요한 테이블: {query_analysis['required_tables']}")
    print(f"필요한 컬럼: {query_analysis['required_columns']}")
    
    # 3. SQL 생성 단계
    print("\n3. SQL 생성 단계")
    sql = sql_generator.generate(query, query_analysis, schema_info)
    print(f"생성된 SQL:\n{sql}")
    
    # 4. SQL 실행 단계
    print("\n4. SQL 실행 단계")
    result = sql_executor.execute(sql)
    
    # 5. 결과 포맷팅 단계
    print("\n5. 결과 포맷팅 단계")
    formatted_result = result_formatter.format(result, query_analysis)
    print(f"최종 결과:\n{formatted_result}")

# 첫 번째 테스트 쿼리로 상세 분석 실행
analyze_agent_workflow(test_queries[0])